In [2]:
import pandas as pd
import numpy as np

In [3]:
data_frame = pd.read_parquet('./dataset/relevance_challenge_valid.parquet')

In [4]:
def softmax(x):
    # Compute the exponential values for each element in the input array
    exps = np.exp(x - np.max(x))

    # Compute the softmax values by dividing the exponential of each element by the sum of exponentials
    return exps / np.sum(exps)  

In [5]:
data_frame['probabilities'] = data_frame['raw_prediction'].apply(softmax)

In [6]:
data_frame['pred'] = data_frame['raw_prediction'].apply(lambda x: np.argmax(x))

In [7]:
x_values = []
y_values = []
counts = []
for j in range(3000):
    filtered_data = data_frame[data_frame['pred'] == j]
    if not filtered_data.empty:
        x_values.append(j)
        y_values.append(filtered_data['distance'].mean())
        counts.append(len(filtered_data['distance']))


In [36]:
mean_distance_pred = pd.DataFrame([x for x in zip(x_values,y_values,counts)])

mean_distance_pred.rename(columns={0: 'feature'}, inplace=True)
mean_distance_pred.rename(columns={1:'mean_distance'},inplace=True)
mean_distance_pred.rename(columns={2:'counts'},inplace=True)

mean_distance_pred_t = mean_distance_pred[mean_distance_pred['counts'] >= 100]
mean_distance_pred_t.describe()

,feature,mean_distance,counts
count,124.000000,124.000000,124.000000
mean,323.475806,3200.795064,273.645161
std,462.741626,1998.821039,248.174447
min,0.000000,493.514270,100.000000
25%,43.500000,1741.740018,129.750000
50%,139.000000,2797.943047,174.000000
75%,438.250000,4310.125920,311.250000
max,2347.000000,10564.948403,1388.000000


In [37]:
valid_pred = mean_distance_pred['feature'].to_list()

In [38]:
data_frame = data_frame[data_frame['pred'].isin(valid_pred)]

In [39]:
bins = [0, 0.1, 0.2, 0.3, 1.0]

probabilities = data_frame['probabilities'].values
probabilities

array([array([7.3262299e-06, 3.1586867e-06, 1.8454843e-06, ..., 3.4390405e-06,
              1.7382778e-06, 1.4658847e-06], dtype=float32)                   ,
       array([1.2205189e-06, 1.4805465e-06, 4.8549100e-06, ..., 9.6833992e-07,
              1.8104382e-06, 1.8591088e-06], dtype=float32)                   ,
       array([4.9958048e-06, 4.3765172e-06, 3.5352220e-06, ..., 4.1862900e-06,
              1.8424200e-06, 1.5242344e-06], dtype=float32)                   ,
       ...,
       array([1.3649012e-03, 5.6243024e-04, 1.2793902e-03, ..., 7.1128510e-05,
              4.2985790e-04, 2.2850082e-04], dtype=float32)                   ,
       array([1.1903144e-03, 7.0683629e-04, 1.3186556e-03, ..., 8.2090621e-05,
              2.4253859e-04, 2.1134102e-04], dtype=float32)                   ,
       array([1.2246306e-03, 1.6635128e-03, 1.6548340e-03, ..., 5.1828687e-05,
              2.4977321e-04, 3.1500243e-04], dtype=float32)                   ],
      dtype=object)

In [40]:
histograms = []
for row in probabilities:
    histograms.append(np.histogram(row, bins=bins)[0])

In [41]:
histograms = np.array(histograms)
histograms = np.delete(histograms, 0, 1)
histograms

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [42]:
column_names = [f'{bins[i]}_{bins[i+1]}' for i in range(len(bins) - 1)]
histograms = histograms.T

In [43]:
data_frame[column_names[1]] = histograms[0]
data_frame[column_names[2]] = histograms[1]
data_frame[column_names[3]] = histograms[2]

In [44]:
data_frame['peaks'] = data_frame['0.1_0.2'] + data_frame['0.2_0.3'] + data_frame['0.3_1.0']
data_frame.sort_values('confidence', ascending=True, inplace=True)
data_frame['distance'].describe()

count    55350.000000
mean      3260.978350
std       4202.391081
min          0.751505
25%        378.965748
50%       1254.280157
75%       4598.103009
max      19791.642460
Name: distance, dtype: float64

In [45]:
data_set = data_frame[['confidence','peaks','distance']]
data_set['features'] = data_set[['confidence' ,'peaks']].values.tolist()
data_set['label'] = data_set['distance'].apply(lambda x: 1/x)

data_set.drop(['confidence','peaks','distance'], axis=1, inplace=True)

/tmp/ipykernel_3880/3143293065.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set['features'] = data_set[['confidence' ,'peaks']].values.tolist()
/tmp/ipykernel_3880/3143293065.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set['label'] = data_set['distance'].apply(lambda x: 1/x)
/tmp/ipykernel_3880/3143293065.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

In [46]:
from sklearn.linear_model import LinearRegression

In [47]:
model = LinearRegression()
model.fit(data_set['features'].tolist(), data_set['label'].tolist())

LinearRegression()

In [48]:
data_frame['score'] = data_frame['confidence'] * model.coef_[0] + data_frame['peaks'] * model.coef_[1]
data_frame.sort_values('score', ascending=False, inplace=True)
data_frame_filtered = data_frame.iloc[0:int(0.1*len(data_frame))]
data_frame_filtered['distance'].describe()

count     5535.000000
mean       655.377445
std       1986.663024
min          2.974029
25%         32.149999
50%         82.422675
75%        297.806394
max      17147.333877
Name: distance, dtype: float64

In [49]:
data_frame_filtered.describe()

,label,confidence,distance,pred,0.1_0.2,0.2_0.3,0.3_1.0,peaks,score
count,5535.000000,5535.000000,5535.000000,5535.000000,5535.000000,5535.000000,5535.000000,5535.000000,5535.000000
mean,1117.669919,0.499479,655.377445,824.029088,0.852575,0.364408,0.867931,2.084914,0.021479
std,868.671302,0.226955,1986.663024,790.498971,1.303556,0.617184,0.550398,1.176975,0.007152
min,1.000000,0.124816,2.974029,1.000000,0.000000,0.000000,0.000000,1.000000,0.012582
25%,312.000000,0.310683,32.149999,208.000000,0.000000,0.000000,1.000000,1.000000,0.015316
50%,928.000000,0.453711,82.422675,531.000000,0.000000,0.000000,1.000000,2.000000,0.019527
75%,1864.500000,0.676044,297.806394,1243.000000,1.000000,1.000000,1.000000,3.000000,0.026776
max,2995.000000,0.989569,17147.333877,2995.000000,6.000000,3.000000,2.000000,6.000000,0.037884
